# IAM Credential Compromise - Detection
This notebook is to be used in case of malicious activity inside your AWS Account.
We will query CloudTrail logs using Athena in order to detect and contain malicious account activity.

We are following best practices from the AWS Incident Response guide as we execute our processes:

https://docs.aws.amazon.com/whitepapers/latest/aws-security-incident-response-guide/aws-security-incident-response-guide.html

# Setup

## Load Libraries

In order to query CloudTrail and interact with AWS, we need to load several libraries and configure our environment.

In [ ]:
pip install pyathena --quiet

In [ ]:
import boto3  # the Python SDK for AWS
import pandas as pd # Pandas is a data analysis tool for Python
from pyathena import connect # Python API client for Amazon Athena
region='us-east-1' # Set region variable to us-east-1 for API commands
athena_bucket = 's3:// ENTER BUCKET NAME HERE /'  # S3 bucket that is configured to store your Athena queries
db_name = 'security_analysis' # database used by Athena. Choose 'default' if your CloudTrail was created via the Athena console and 'security_analysis' if you are using the AWS Security Analytics Bootstrap

## 1.0 Set up helper function for Athena

The Python query_results function shown below will help you query Athena tables.

In [ ]:
def query_results(sql):
    
    cursor = connect(s3_staging_dir=athena_bucket, region_name=region).cursor()
    cursor.execute(sql)
    
    columns = cursor.description
    data = cursor.fetchall()

    column_names = [column[0] for column in columns]
    rows = [dict(zip(column_names, data_row)) for data_row in data]

    df = pd.DataFrame(rows, columns=column_names)
    df1 = df.style.set_table_styles([dict(selector='th', props=[('text-align', 'center')])])
    
    return df1.set_properties(**{'text-align': 'center'})

# 2.0 IAM Investigation

### 2.1 Investigating Authorization Failures
Lists failed access attempts with additional information such as the user identity ARN, time, AWS service, API call, region, source IP address, error code, and error message.

In [ ]:
auth_fail =f"""

select *
from "{db_name}"."cloudtrail" 
where errorCode in 
('Client.UnauthorizedOperation','Client.InvalidPermission.NotFound','Client.OperationNotPermitted','AccessDenied')
and useridentity.arn like '%iam%'
order by eventTime desc
limit 10

"""

In [ ]:
results = query_results(auth_fail)
results

### 2.2 Investigating the User Identities with Authorization Failures
As you know there are some failed access attempts, lets now look at the user identities with a lot of authorization failures.

In [ ]:
usr_auth_fail =f"""

select userIdentity.arn, count(*) as total
from "{db_name}"."cloudtrail" 
where errorCode in ('Client.UnauthorizedOperation','Client.InvalidPermission.NotFound','Client.OperationNotPermitted','AccessDenied')
and useridentity.arn like '%iam%'
group by userIdentity.arn
order by total desc

"""

In [ ]:
results = query_results(usr_auth_fail)
results

### 2.3 Investigating an IAM User
Now that you have identified the potential IAM entity which has been compromised, you should do further investigations to identify what the entity has been attemping to do.

In [ ]:
iam_usr =f"""

select *
from "{db_name}"."cloudtrail" 
where userIdentity.arn='insert ARN of IAM Credential from query in 2.2'
limit 5

"""

In [ ]:
results = query_results(iam_usr)
results

### 2.4 Investigation the AWS services Used by the Suspicious Identity
Now that you have identified the potential IAM entity which has been compromised, you should do further investigations to identify what services the suspicious identity has been attempting to use.

In [ ]:
services_used =f"""

select eventSource, count(*) as total
from "{db_name}"."cloudtrail" 
where userIdentity.arn='insert ARN of IAM Credential from query in 2.2'
group by eventSource
order by total desc

"""

In [ ]:
results = query_results(services_used)
results

### 2.5 Investigation the Action Attempted by the Suspicious Identity
Now that you have identified the potential IAM entity which has been compromised, you should do further investigations to identify what action the suspicious identity has been attempting to take.

In [ ]:
act_attempt =f"""
select eventName, count(*) as total
from "{db_name}"."cloudtrail" 
where userIdentity.arn='insert ARN of IAM Credential from query in 2.2'
group by eventName
order by total desc

"""

In [ ]:
results = query_results(act_attempt)
results

### 2.6 Bringing it all together
Bring together the previous queries to create a single query showing the event name, AWS service, and AWS region where requests were being made to by the compromised IAM entity.

In [ ]:
service_apicalls_time =f"""

select eventTime, eventSource, eventName, awsregion, count(*) AS total 
from "{db_name}"."cloudtrail" 
where userIdentity.arn='insert ARN of IAM Credential from query in 2.2'
group by eventName, eventSource, eventTime, awsregion
order by eventTime DESC

"""

In [ ]:
results = query_results(service_apicalls_time)
results

### Title of the query
Insert description about the purpose and expected output of the query

In [ ]:
query =f"""

"""

In [ ]:
results = query_results(query)
results

## [Credential Compromise Containment Notebook](./credential-compromise-containment.ipynb)